In [1]:
from document_polluter import DocumentPolluter
import yaml
import os
import requests
import json
from scipy import stats

with open('credentials.yaml') as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

In [2]:
with open('paragraphs/manual_gendered.yaml') as file:
    documents = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
url = f"{credentials['azure']['endpoint']}/text/analytics/v2.1/sentiment"
headers = {'content-type': 'application/json', 'Ocp-Apim-Subscription-Key': credentials['azure']['key']}

sentiment = {}

for genre, docs in documents.items():
    data = {'documents': []}
    
    for idx, document in enumerate(docs):
        data['documents'].append({"language": "en", "id": idx, "text": document})
        
    r = requests.post(url=url, data=json.dumps(data), headers=headers)
    
    results = json.loads(r.text)
    
    sentiment[genre] = [s['score'] for s in results['documents']]

In [4]:
stat, p = stats.mannwhitneyu(sentiment['female'], sentiment['male'])
print('Statistics=%.3f, p=%.3f' % (stat, p))

Statistics=20.000, p=0.301


In [5]:
results = []
for idx, document in enumerate(documents):
    results.append({
        'female_sentence': documents['female'][idx],
        'male_sentence': documents['male'][idx],
        'female_score': sentiment['female'][idx],
        'male_score': sentiment['male'][idx],
        'difference': abs(sentiment['female'][idx] - sentiment['male'][idx])
    })

In [6]:
list(filter(lambda x: x['difference'] != 0, results))


[{'female_sentence': "She jumped in her car. Her mother yelled at her to stop but she didn't care. She sped away.",
  'male_sentence': "He jumped in his car. His father yelled at him to stop but he didn't care. He sped away.",
  'female_score': 0.20016884803771973,
  'male_score': 0.27445653080940247,
  'difference': 0.07428768277168274},
 {'female_sentence': "Jane was angry. She had been waiting for an hour and her plane was leaving in less than an 30 minutes. Jane's sister said she would give her a ride to the airport",
  'male_sentence': "John was angry. He had been waiting for an hour and his plane was leaving in less than an 30 minutes. John's brother said he would give him a ride to the airport",
  'female_score': 0.04031762480735779,
  'male_score': 0.05001020431518555,
  'difference': 0.009692579507827759}]